In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix, classification_report
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# report_filename = 'model_100_epochs_evaluation.csv'
# report_filename = 'model_500_samples_paper_evaluation.csv'
report_filename = 'model_500_samples_paper_v5_evaluation.csv'


report_path = os.path.join('..', 'analysis', 'finetuned', report_filename)
df = pd.read_csv(report_path).set_index('filepath')

In [3]:
available_classes = list(df['label'].unique())

available_classes = available_classes[::-1]


In [4]:
df.columns

Index(['label', 'concrete_plates', 'paving_stones', 'sett', 'asphalt', 'grass',
       'compacted', 'concrete', 'cobblestone', 'gravel', 'ground'],
      dtype='object')

In [5]:
predicted_labels = list(df[available_classes].idxmax(axis=1))

In [6]:
true_labels = list(df['label'])

In [7]:
assert len(predicted_labels) == len(true_labels)

In [8]:
c_mat = confusion_matrix(true_labels,
                         predicted_labels,
                         labels=available_classes,
                         normalize='true') * 100

In [9]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes,
                y=available_classes,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [10]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes,
    output_dict=True
)

{'sett': {'precision': 0.8877551020408163,
  'recall': 0.87,
  'f1-score': 0.8787878787878788,
  'support': 200.0},
 'paving_stones': {'precision': 0.9257425742574258,
  'recall': 0.935,
  'f1-score': 0.9303482587064676,
  'support': 200.0},
 'ground': {'precision': 0.9848484848484849,
  'recall': 0.975,
  'f1-score': 0.9798994974874372,
  'support': 200.0},
 'gravel': {'precision': 0.97,
  'recall': 0.97,
  'f1-score': 0.97,
  'support': 200.0},
 'grass': {'precision': 0.9950248756218906,
  'recall': 1.0,
  'f1-score': 0.9975062344139651,
  'support': 200.0},
 'concrete_plates': {'precision': 0.949238578680203,
  'recall': 0.935,
  'f1-score': 0.9420654911838791,
  'support': 200.0},
 'concrete': {'precision': 0.8756218905472637,
  'recall': 0.88,
  'f1-score': 0.8778054862842892,
  'support': 200.0},
 'compacted': {'precision': 0.9251336898395722,
  'recall': 0.865,
  'f1-score': 0.8940568475452196,
  'support': 200.0},
 'cobblestone': {'precision': 0.95,
  'recall': 0.95,
  'f1-scor

In [11]:
c_mat.diagonal().sum() / (len(available_classes))

93.5